# Semantic Reasoning on Geospatial Entities in an Archaeological Dataset

The following python code runs SPARQL queries on data in the Kenchreai Archaeological Archive (http://kenchreai.org/kaa). It moves towards semantic reasoning that lists lamps found at the site that were produced in the Central Mediterranean. Note that individual lamps do not directly state their region of production. That information is associated with the lamp type and applied to each lamp by OWL-based reasoning.

In [17]:
# core modules
import os
import re
import urllib.parse
import urllib.request

# other modules
from bs4 import BeautifulSoup

import dominate
from dominate.tags import *

from IPython.display import display
from IPython.display import Image
from IPython.display import HTML

from SPARQLWrapper import SPARQLWrapper, JSON

In [18]:
endpoint = SPARQLWrapper("http://kenchreai.org/endpoint/kenchreai/query")
endpoint.setReturnFormat(JSON)

reasoner = SPARQLWrapper("http://kenchreai.org/reasoner/kenchreai/query")
reasoner.setReturnFormat(JSON)


In [ ]:
# start with a simple SPARQL query to the non-reasoning endpoint
endpoint.setQuery('SELECT * WHERE { kaa:central-mediterranean ?p ?o }')
kaaresult = endpoint.query().convert()

# if you want to see the raw JSON resut, uncomment next line.
# kaaresult

In [20]:
# loop through json result and print out predicate and object for kaatyp:
for result in kaaresult["results"]["bindings"]:
    print('%s %s' % (result['p']['value'],result['o']['value']))

# depending on your jupyter defaults, links may be clickable.

http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://kenchreai.org/kaa/ontology/geographic-entity
http://www.w3.org/2000/01/rdf-schema#label Central Mediterranean
http://kenchreai.org/kaa/ontology/is-logical-part-of http://kenchreai.org/kaa/geographic-entities


In [21]:
# find and list typology items that point to kaa:central-mediterranean
# as a broader concept.
endpoint.setQuery('''SELECT ?uri WHERE {

 ?uri kaaont:broader kaa:central-mediterranean

}''')
broader = endpoint.query().convert()

for result in broader["results"]["bindings"]:
    print('%s' % (result['uri']['value']))
    
# again, links may be clickable

http://kenchreai.org/kaa/italy
http://kenchreai.org/kaa/north-africa
http://kenchreai.org/kaa/tripolitania


In [22]:
# the above SPARQL queries don't use semantic reasoning.
# the definition of kaaont:broader includes the triple
# "kaatyp:visual-motif-christian-symbol owl:inverseOf kaaont:narrower ."

reasoner.setQuery('''SELECT ?uri WHERE {

 kaa:central-mediterranean kaaont:narrower ?uri

}''')
broader = reasoner.query().convert()

for result in broader["results"]["bindings"]:
    print('%s' % (result['uri']['value']))

# the resulting list should be the same as the produced by the previous cell.

http://kenchreai.org/kaa/italy
http://kenchreai.org/kaa/north-africa
http://kenchreai.org/kaa/tripolitania


In [23]:
reasoner.setQuery('''SELECT ?item WHERE {

 ?item kaaont:is-part-of+ kaa:central-mediterranean ; 
                        a kaaont:inventory-number .
                    

} ORDER BY ?item''')

items = reasoner.query().convert()

for result in items["results"]["bindings"]:
    print('%s' % (result['item']['value']))

# this list is all the inventoried objects in KAA said directly or indirectly to be from the "Central Mediterrean" (CM).
# Click on one (or otherwise go to that address). If it doesn't directly state a geographic origin, click on "Typological
# Identification". That should put you on track to discovering why the obect has a CM origin.

http://kenchreai.org/kaa/kcp/kp1989-008
http://kenchreai.org/kaa/kcp/kp1990-005
http://kenchreai.org/kaa/ke/ke0019
http://kenchreai.org/kaa/ke/ke0033
http://kenchreai.org/kaa/ke/ke0110
http://kenchreai.org/kaa/ke/ke0113
http://kenchreai.org/kaa/ke/ke0342
http://kenchreai.org/kaa/ke/ke0366
http://kenchreai.org/kaa/ke/ke0369
http://kenchreai.org/kaa/ke/ke0397
http://kenchreai.org/kaa/ke/ke0408
http://kenchreai.org/kaa/ke/ke0411
http://kenchreai.org/kaa/ke/ke0440
http://kenchreai.org/kaa/ke/ke0501
http://kenchreai.org/kaa/ke/ke0518
http://kenchreai.org/kaa/ke/ke0519
http://kenchreai.org/kaa/ke/ke0543
http://kenchreai.org/kaa/ke/ke0546
http://kenchreai.org/kaa/ke/ke0602
http://kenchreai.org/kaa/ke/ke0642
http://kenchreai.org/kaa/ke/ke0643
http://kenchreai.org/kaa/ke/ke0644
http://kenchreai.org/kaa/ke/ke0646
http://kenchreai.org/kaa/ke/ke0671
http://kenchreai.org/kaa/ke/ke0705
http://kenchreai.org/kaa/ke/ke0741
http://kenchreai.org/kaa/ke/ke0764
http://kenchreai.org/kaa/ke/ke0775
http://ken

In [32]:
# look for lamps from the CM

reasoner.setQuery('''SELECT ?kaauri ?label WHERE {

 ?kaauri kaaont:is-part-of+ kaa:central-mediterranean ; 
                        kaaont:typological-identification/kaaont:broader* kaatyp:l ;
                        rdfs:label ?label.
} ORDER BY ?kaauri''')

items = reasoner.query().convert()

# Use the iPython rich display module to make nicer html: label with links
h = '<h3>A list of Lamps from the Central Mediterranean Found at Kenchreai</h3>'
for result in items["results"]["bindings"]:
    h += '<p><a href="%s">%s</a></p>' % (result['kaauri']['value'],result['label']['value'])
HTML(h)